# What kind of a dog is this?
Final project for Data Science bootcamp organized by Sages.

Importing necessary modules

In [10]:
import os
import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd
import scipy.io as sio
import imageio
from skimage.transform import resize
from tqdm import tqdm_notebook
from pickle import dump

from xml.dom import minidom
from tqdm import tqdm_notebook

from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.resnet50 import preprocess_input, decode_predictions

Downloading the data from the website

In [2]:
#TODO: code for downloading the data from the website

Loading the data

Initzialization of base global variables

In [4]:
base_folder = 'dataset'
images_path = os.path.join(base_folder, 'Images')
annots_path = os.path.join(base_folder, 'Annotation')
cropps_path = os.path.join(base_folder, 'Cropped')
image_size = (224, 224)

In [5]:
def crop_images(images_path):
    total_folders = sum(1 for e in os.scandir(images_path) if e.is_dir())
    for entry in tqdm_notebook(os.scandir(images_path), total=total_folders, unit="folders"):
        if not entry.name.startswith('.') and entry.is_dir():
            crop_folder_images(entry.path)

def crop_folder_images(folder_path):
    for entry in os.scandir(folder_path):
        if not entry.name.startswith('.') and entry.is_file():
            crop_image(entry.path)
                    
def crop_image(image_path, output_folder = 'dataset/Cropped'):
    try:
        image_data = imageio.imread(image_path)

        annon_xml = minidom.parse(image_path.replace("Images", "Annotation")[:-4])
        
        xmin = int(annon_xml.getElementsByTagName('xmin')[0].firstChild.nodeValue)
        ymin = int(annon_xml.getElementsByTagName('ymin')[0].firstChild.nodeValue)
        xmax = int(annon_xml.getElementsByTagName('xmax')[0].firstChild.nodeValue)
        ymax = int(annon_xml.getElementsByTagName('ymax')[0].firstChild.nodeValue)

        new_image_data = image_data[ymin:ymax,xmin:xmax,:]
        new_image_data = resize(new_image_data, image_size, preserve_range=True).astype('uint8')
        
        output_folder_path = os.path.join(output_folder, image_path.split('/')[-2])
        image_name = image_path.split('/')[-1]
        
        if not os.path.exists(output_folder_path):
            os.makedirs(output_folder_path)
        
        imageio.imwrite(
            os.path.join(output_folder_path, image_name), 
            new_image_data
        )
        
    except IOError as e:
        print('Could not read: {}: {} - skipping.'.format(image_path, e))

In [6]:
crop_images(images_path)

Could not read: dataset/Images/n02105855-Shetland_sheepdog/n02105855_2933.jpg: JPEG does not support alpha channel. - skipping.



We will use ResNet as a base model

In [7]:
model = ResNet50(weights='imagenet')
# re-structure the model
model.layers.pop()
model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
# summarize
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [18]:
def extract_features(base_path):
    features = dict()
    
    files_list = [os.path.join(dirpath, filename) 
                  for dirpath, dirs, filenames in os.walk(base_path) 
                  for filename in filenames if not filename.startswith('.')
                 ]
    
    #temp variables for batch dumping features to file
    #and keeping track of already processed files 
    i = 0
    processed_files = []
    for filename in tqdm_notebook(files_list):
        if not filename.startswith('.'):
            # load an image from file
            image = load_img(filename, target_size=image_size)
            
    dump(features, open('temp_features.pkl', 'wb'))
    
    return features, processed_files

In [19]:
features, processed_files = extract_features(cropps_path)
print('Extracted Features: %d' % len(features))
# save to file
dump(features, open('features.pkl', 'wb'))

Extracted Features: 0


# References


Aditya Khosla, Nityananda Jayadevaprakash, Bangpeng Yao and Li Fei-Fei.**Novel dataset for Fine-Grained Image Categorization.**_First Workshop on Fine-Grained Visual Categorization (FGVC), IEEE Conference on Computer Vision and Pattern Recognition (CVPR)_, 2011.  [\[pdf\]](http://people.csail.mit.edu/khosla/papers/fgvc2011.pdf)  [\[poster\]](http://vision.stanford.edu/documents/KhoslaJayadevaprakashYaoFeiFei_FGVC2011.pdf)  [\[BibTex\]](http://vision.stanford.edu/bibTex/KhoslaJayadevaprakashYaoFeiFei_FGVC2011.bib)

J. Deng, W. Dong, R. Socher, L.-J. Li, K. Li and L. Fei-Fei,**ImageNet: A Large-Scale Hierarchical Image Database.**_IEEE Computer Vision and Pattern Recognition (CVPR)_, 2009.  [\[pdf\]](http://www.image-net.org/papers/imagenet_cvpr09.pdf)  [\[BibTex\]](http://www.image-net.org/papers/imagenet_cvpr09.bib)